## Quantum adder

In [1]:
from projectq import MainEngine
from projectq.backends import CircuitDrawer
from projectq.ops import X, CNOT, Measure, All, ControlledGate

def qsum_(c, a, b):
    CNOT|(a, b)  # CNOT (control bit, target bit)
    CNOT|(c, b)
    return b       # b -> b' = a+b+c mod 2

<img src="Images/sum.png" width=300>

In [2]:
def qcarry_(eng, a, b, c):
    ancilla = eng.allocate_qubit()
    ControlledGate(X,2) | (a,b,ancilla) # ControlledGate(gate, 2) | (qb0, qb2, qb3), qb0 & qb2 are controls
    CNOT|(a,b)
    ControlledGate(X,2) | (c,b,ancilla)
    return (c,a,b,ancilla)  # (carry, a, a+b mod 2, next carry)
                            # a+b mod 2 can be uncomputed using CNOT

<img src="Images/carry.png" width=300>

In [3]:
def qadd_(eng, qureg_A, qureg_B):
    assert len(qureg_A)==len(qureg_B)
    n=len(qureg_A)
    carries = [None for i in range(n+1)]
    carries[0] = eng.allocate_qubit()
    for i in range(n):
        _, _, _, carries[i+1] = qcarry_(eng,qureg_A[i],qureg_B[i],carries[i])
        CNOT|(qureg_A[i],qureg_B[i])      # uncompute a+b mod 2 to b
        qsum_(carries[i],qureg_A[i],qureg_B[i])
        
    All(Measure)|qureg_B # less significant bits
    Measure|carries[n]   # most significant bit
    eng.flush()
    
    result = ''
    for bit in qureg_B:
        result += str(int(bit))
    result += str(int(carries[n]))
    return result[::-1] # reverse